# Imports

In [72]:
from ROOT import TFile, TTree
import numpy as np
from array import array

%jsroot on

# Load data

In [4]:
fin = TFile("/data_CMS/cms/kalipoliti/chargedSJ_mergedSVtracks_gen_reco/merged_HiForestAOD.root")
t = fin.Get("ak4PFJetAnalyzer/t")
t.AddFriend("hiEvtAnalyzer/HiTree") # to get the evt weight

<ROOT.TFriendElement object ("hiEvtAnalyzer/HiTree") at 0xa9f42d0>

In [18]:
t.Show(1)

======> EVENT:1
 nref            = 2
 rawpt           = 32.4949, 
                  20.732
 jtpt            = 35.5583, 
                  23.526
 jteta           = 0.81085, 
                  -1.02337
 jtphi           = 2.76803, 
                  -0.151324
 jtHadFlav       = 5, 
                  0
 jtParFlav       = -5, 
                  21
 jtNbHad         = 1, 
                  0
 jtNcHad         = 2, 
                  0
 jtNbPar         = 1, 
                  0
 jtNcPar         = 0, 
                  0
 jtHasGSPB       = 0, 
                  0
 jtHasGSPC       = 0, 
                  0
 jtIsHardest     = 0, 
                  0
 sjt1E           = 29.7446, 
                  12.2204
 sjt1Pt          = 21.8163, 
                  7.6736
 sjt1Eta         = 0.824327, 
                  -1.03665
 sjt1Phi         = 2.7944, 
                  -0.0854072
 sjt1HadFlav     = 0, 
                  0
 sjt1HadFlav     = 0, 
                  0
 sjt2E           = 3.79889, 
               

In [22]:
t.GetEntry(0)
for ijet in range(t.nref):
    nsv = t.nsvtx[ijet]
    print(nsv)

0
0
1
1


In [71]:
t.GetEntry(213)
njet = t.nref
l = []
for ijet in range(njet):
    #l.append(np.array(t.svtxTrEta[ijet]))
    nsv = t.nsvtx[ijet]
    for isv in range(nsv):
        if (t.svtxntrk[ijet][isv] > 0):
            print(np.array(t.svtxTrEta[ijet]))
#print(np.concatenate(l))

[1.4006848 1.4247342 1.370298  1.382599 ]
[0.8662487]


In [80]:
sum(x for x in [-1,0,1,2])

2

In [105]:
nSvTrk = sum(ntrk for ntrk in np.ravel(t.svtxntrk) if ntrk > 0)

In [88]:
nSvTrk

5

In [104]:
t.svtxntrk

<ROOT.vector<vector<int> > object at 0xaaaba60>

In [91]:
np.ravel(t.svtxntrk)

array([4, 1, 0], dtype=int32)

In [77]:
x = 0
sum(np.array(t.svtxntrk))

array([5], dtype=int32)

In [97]:
t.svtxTrEta

<ROOT.vector<vector<float> > object at 0xaabd460>

In [95]:
np.array(t.svtxTrEta, dtype = float)

ValueError: setting an array element with a sequence.

In [93]:
np.ravel(t.svtxTrEta)

array([<ROOT.vector<float> object at 0x156bb180>,
       <ROOT.vector<float> object at 0x156bb198>,
       <ROOT.vector<float> object at 0x156bb1b0>], dtype=object)

In [115]:
len(t.svtxTrEta[2])

0

In [118]:
np.concatenate([np.array(t.svtxTrEta[ijet]) for ijet in range(njet) if (len(t.svtxTrEta[ijet]) > 0)])

array([1.4006848, 1.4247342, 1.370298 , 1.382599 , 0.8662487],
      dtype=float32)

In [62]:
np.concatenate([np.array(t.svtxTrEta[ijet]) for ijet in range(njet)])

ValueError: setting an array element with a sequence.

# Create histograms

In [137]:
ts = [0,1,1]
l = np.concatenate([np.array([ts[i]]) if ts[i]>0 else [] for i in ts])
l

array([1., 1.])

In [135]:
sum(i if i > 0 else 10 for i in ts)

12

In [57]:
from ROOT import TH1D, TCanvas

Count the percentage of tracks coming from B decay products that are associated to a secondary vertex (SV).

In [74]:
t.nref

3

In [ ]:
def flatten(tree, entry, branch):
    tree.GetEntry(entry)
    nref = tree.nref
    
    
    for ijet in range(nref):
        nsv = tree.nsvtx[ijet]
        for isv in range(nsv):
            if (t.svtxntrk[ijet][isv] > 0):

In [124]:
hsv = TH1D("hsv", "Percentage of tracks from B products associated to any SV per event", 25, 0, 100)
hsv.SetXTitle("% b-product tracks in SVs per event")
hsv.SetYTitle("nb of events")

nentries = t.GetEntries()

for ient in range(nentries):
    ient = int(ient)
    
    # for debugging purposes 
    if (ient < 30000): 
        continue
    if (ient > 40000):
        break
    
    t.GetEntry(ient)
    
    bProducts = 0
    bProductsInSV = 0
    for itrack in range(t.nIP):
        itrack = int(itrack)
        
        # Keep only B decay products
        sta = t.ipMatchStatus[itrack]
        if (sta < 100): 
            continue
        bProducts += 1
        
        # Look for track in SV
        eps = 1e-3
        
        trackFoundInSV = False
        svtxTrEta = np.concatenate([np.array(t.svtxTrEta[ijet]) for ijet in range(t.nref) if (len(t.svtxTrEta[ijet]) > 0)])
        svtxTrPhi = np.concatenate([np.array(t.svtxTrPhi[ijet]) for ijet in range(t.nref) if (len(t.svtxTrPhi[ijet]) > 0)])
        for svEta, svPhi in zip(svtxTrEta, svtxTrPhi):
            if ((t.ipEta[itrack] - svEta) > eps): 
                continue
            if ((t.ipPhi[itrack] - svPhi) > eps): 
                continue
            trackFoundInSV = True
            break
            
        if trackFoundInSV:
            bProductsInSV += 1
    perc = -1
    if (bProducts > 0):
        perc = (bProductsInSV / bProducts) * 100
    hsv.Fill(perc, t.weight)       
    
c = TCanvas("c", "c", 800, 800)
hsv.Draw("hist")
c.Draw()

KeyboardInterrupt: 

Warning in <TFile::Append>: Replacing existing TH1: hsv (Potential memory leak).
